<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [6]</a>'.</span>

In [1]:
import time
import numpy as np
import types
import os
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d
from scipy.special import zeta
import PRyM.PRyM_init as PRyMini
import PRyM.PRyM_thermo as PRyMthermo
import PRyM.test_main3 as test_main
#import PRyM.PRyM_jl_sys as PRyMjl
from numdifftools import Derivative
import PRyM.PRyM_eval_nTOp as PRyMevalnTOp
import PRyM.PRyM_nTOp as PRyMnTOp
import matplotlib.pyplot as plt
import pandas as pd
import random

In [2]:
#observed values, all wrong currently, for testing purposes
#Neff_obs, Omeganurel_obs, OneOverOmeganunr_obs, YPCMB_obs, YPBBN_obs, DoHx1e5_obs, He3oHx1e5_obs, Li7oHx1e10_obs = np.array([3.04439907, 5.69942692, 93.03336616, 0.24552507, 0.24685084, 2.5276775, 1.04572838, 5.27256451])
#Yp, Omega_B h^2, Neff
#YPCMB_obs, OmegaB_obs, Neff_obs, DoHx1e5_obs, Li7oHx1e10_obs
YPCMB_obs, Neff_obs, DoHx1e5_obs, Li7oHx1e10_obs = np.array([0.24,3.85,2.547,1.63])
X_obs = np.array([YPCMB_obs, Neff_obs, DoHx1e5_obs, Li7oHx1e10_obs])
#X_obs = np.array([Neff_obs, Omeganurel_obs, OneOverOmeganunr_obs, YPCMB_obs, YPBBN_obs, DoHx1e5_obs, He3oHx1e5_obs, Li7oHx1e10_obs])
#Neff_sig, Omeganurel_sig, OneOverOmeganunr_sig, YPCMB, YPBBN_sig, DoHx1e5_sig, He3oHx1e5_sig, Li7oHx1e10_sig = np.array([0.5,0.5,3,.1,.1,.3,.2,1])
YPCMB_sig, Neff_sig, DoHx1e5_sig, Li7oHx1e10_sig = np.array([0.24,0.62,0.029,0.3])
#sigma_X = np.array([Neff_sig, Omeganurel_sig, OneOverOmeganunr_sig, YPCMB, YPBBN_sig, DoHx1e5_sig, He3oHx1e5_sig, Li7oHx1e10_sig])
sigma_X = np.array([YPCMB_sig, Neff_sig, DoHx1e5_sig, Li7oHx1e10_sig])

In [3]:
'''filename = "output.txt"

# Create the file (or clear it if it already exists)
with open(filename, 'w') as f:
    f.write("Starting output:\n")'''

'filename = "output.txt"\n\n# Create the file (or clear it if it already exists)\nwith open(filename, \'w\') as f:\n    f.write("Starting output:\n")'

In [4]:
# Chi^2 function
#params = [gamma,delta,alpha,delm,m0,fplas,f,Tend,Nspecies]
def chi2_fn(params):
    #initialize stasis parameters
    PRyMini.compute_bckg_flag = True
    PRyMini.save_bckg_flag = False
    PRyMini.stasis_flag = True
    PRyMini.verbose_flag = False
    PRyMini.stasis_params['gamma'] = params[0]
    PRyMini.stasis_params['delta'] = params[1]
    #PRyMini.stasis_params['alpha'] = params[2]
    PRyMini.stasis_params['alpha'] = (2/7)*PRyMini.stasis_params['gamma'] - (1/PRyMini.stasis_params['delta'])
    PRyMini.stasis_params['Delta_m'] = params[3]
    PRyMini.stasis_params['m0'] = params[4]
    PRyMini.stasis_params['plasma_inject_frac'] = params[5]
    PRyMini.stasis_params['tiny_radiation'] = params[6]
    PRyMini.stasis_params['stasis_end_mev'] = params[7]
    PRyMini.stasis_params['N_SPECIES'] = int(params[8])
    #print(PRyMini.stasis_params)
    #predictions = model(params)
    #residuals = (predictions - X_obs) / errors
    #sum(((X_th-X_obs)/sigma_X)**2)
    #calculate theoretical values
    results = test_main.PRyMclass().PRyMresults()
    Neff, Omeganurel, OneOverOmeganunr, YPCMB, YPBBN, DoHx1e5, He3oHx1e5, Li7oHx1e10 = results
    #X_th = np.array([Neff, Omeganurel, OneOverOmeganunr, YPCMB, YPBBN, DoHx1e5, He3oHx1e5, Li7oHx1e10])
    X_th = np.array([YPCMB, Neff, DoHx1e5, Li7oHx1e10])
    return sum(((X_th-X_obs)/sigma_X)**2)

# Gradient of chi^2 (numerical)
def grad_chi2(params, eps=1e-5):
    grad = np.zeros_like(params)
    for i in range(len(params)):
        dparams = np.zeros_like(params)
        dparams[i] = eps
        grad[i] = (chi2_fn(params + dparams) - chi2_fn(params - dparams)) / (2 * eps)
    return grad

# Gradient descent routine
def gradient_descent(init_params, lr=0.01, tol=1e-6, max_iter=100):
    params = np.array(init_params)#, dtype=float)
    '''with open(filename, 'a') as f:
        f.write(str(params))
        f.flush()'''
    #for step in range(max_iter):
    #print(step)
    grad = grad_chi2(params)
    print('gradient: ',grad)
    print('params_init: ',init_params)
    #update params
    for i in range(len(params)):
        #gamma
        if i==0:
            if params[i] - lr * grad[i]>0 and params[i] - lr * grad[i]<10:
                params[i] -= lr * grad[i]
            else:
                params[i] = params[i]
        #delta
        if i==1:
            if params[i] - lr * grad[i]>0 and params[i] - lr * grad[i]<10:
                params[i] -= lr * grad[i]
            else:
                params[i] = params[i]
        #alpha
        #alpha is determined from delta and gamma
        '''if i==2:
            if in params[i] - lr * grad[i]>0:
                params[i] -= lr * grad[i]
            else:
                params[i] = params[i]'''
        #delta_m
        if i==3:
            if params[i] - lr * grad[i]>0:
                params[i] -= lr * grad[i]
            else:
                params[i] = params[i]
        #m0
        if i==4:
            params[i]=params[i-1]
            '''if in params[i] - lr * grad[i]>0:
                params[i] -= lr * grad[i]
            else:
                params[i] = params[i]'''
        #plasma_inject_frac
        if i==5:
            if params[i] - lr * grad[i]>0 and params[i] - lr * grad[i] < 1:
                params[i] -= lr * grad[i]
            else:
                params[i] = params[i]
        #tiny_radiation
        if i==6:
            if params[i] - lr * grad[i]>0 and params[i] - lr * grad[i] < 1:
                params[i] -= lr * grad[i]
            else:
                params[i] = params[i]
        #Tend
        if i==7:
            if params[i] - lr * grad[i]>0:
                params[i] -= lr * grad[i]
            else:
                params[i] = params[i]
        #Nspecies
        if i==8:
            if params[i] - lr * grad[i]>0:
                params[i] -= lr * grad[i]
                params[i] = int(np.floor(params[i]))
            else:
                params[i] = int(params[i])

    PRyMini.stasis_params['gamma'] = params[0]
    PRyMini.stasis_params['delta'] = params[1]
    #PRyMini.stasis_params['alpha'] = params[2]
    PRyMini.stasis_params['alpha'] = (2/7)*PRyMini.stasis_params['gamma'] - (1/PRyMini.stasis_params['delta'])
    params[2] = (2/7)*PRyMini.stasis_params['gamma'] - (1/PRyMini.stasis_params['delta'])
    PRyMini.stasis_params['Delta_m'] = params[3]
    PRyMini.stasis_params['m0'] = params[4]
    PRyMini.stasis_params['plasma_inject_frac'] = params[5]
    PRyMini.stasis_params['tiny_radiation'] = params[6]
    PRyMini.stasis_params['stasis_end_mev'] = params[7]
    PRyMini.stasis_params['N_SPECIES'] = params[8]
    #params -= lr * grad
    '''with open(filename, 'a') as f:
        f.write(str(params))
        f.flush()'''

    '''if np.linalg.norm(grad) < tol:
        print(f"Converged in {step} steps.")
        break'''
    print('params_fin: ',params)
    return params

In [5]:
PRyMini.stasis_params

{'gamma': 7,
 'delta': 1,
 'm0': 1.0,
 'Delta_m': 1.0,
 'N_SPECIES': 10,
 'GAMMA_OVER_H': 100000.0,
 'stasis_end_mev': 50,
 'tiny_radiation': 0.01,
 'plasma_inject_frac': 0.1,
 'alpha': 1.0}

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [6]:
#run stasis gradient descent and save results ot a file
# Optional: write header only once
file_name = "results_grad_descent.txt"
with open(file_name, "w") as ff:
    ff.write("gamma delta alpha delm m0 fplas f Tend Nspecies chi2\n")
#initialize random stasis parameters in the desired intervals
gamma = random.uniform(4,8)
delta = random.uniform(0,3)
alpha = (2/7)*gamma - (1/delta)
delm = random.uniform(1,10)
m0 = delm
fplas = random.uniform(0.1,1)
f = random.uniform(0.1,1)
Tend = random.uniform(1, 10)
Nspecies = int(random.randint(1, 20))
#PRyMini.stasis_params
#print(Nspecies)
#initialize params
params = [gamma,delta,alpha,delm,m0,fplas,f,Tend,Nspecies]
chi2 = chi2_fn(params)
with open(file_name, "a") as ff:
    ff.write(f"{gamma} {delta} {alpha} {delm} {m0} {fplas} {f} {Tend} {Nspecies} {chi2}\n")
#apply gradient descent, extremely slow right now
for n in range(1000):
    params = gradient_descent(params, lr=0.001, tol=7, max_iter=100)
    chi2 = chi2_fn(params)
    with open(file_name, "a") as ff:
        ff.write(f"{params[0]} {params[1]} {params[2]} {params[3]} {params[4]} {params[5]} {params[6]} {params[7]} {params[8]} {chi2}\n")
        ff.flush()

gradient:  [-4743.65928377 30956.27152887     0.         71107.37131414
 18917.51513776 16412.35183269 17764.28706198 -5756.63472208
 -5912.47386647]
params_init:  [5.991717857255438, 1.1650518340105016, 0.8535884785455198, 5.556582254890086, 5.556582254890086, 0.9034122984533277, 0.8381547709187545, 5.674058180255756, 19]
params_fin:  [ 5.99171786  1.16505183  0.85358848  5.55658225  5.55658225  0.9034123
  0.83815477 11.4306929  24.        ]


gradient:  [ 30181.34103549 -30142.69531546      0.           4903.63498895
   6576.44634642  -7146.76297981  37923.71812043    375.73098325
  -8241.94400545]
params_init:  [ 5.99171786  1.16505183  0.85358848  5.55658225  5.55658225  0.9034123
  0.83815477 11.4306929  24.        ]
params_fin:  [ 5.99171786  1.16505183  0.85358848  0.65294727  0.65294727  0.9034123
  0.83815477 11.05496192 32.        ]


/dfs6/pub/ahewitt1/Stasis/Stasis_prym/PRyM/test_main3.py:967: RuntimeWarning: overflow encountered in exp
  dchi_g = -4.*H + np.sum(self.stasis["Gamma_ell_s"] * np.exp(chi_ell - chi_g))
/dfs6/pub/ahewitt1/Stasis/.venv/lib64/python3.11/site-packages/scipy/integrate/_ivp/common.py:366: RuntimeWarning: invalid value encountered in multiply
  update = max_diff[ind] * scale_new < max_diff_new * scale[ind]


ValueError: array must not contain infs or NaNs

In [ ]:
#minimize -2 ln L with a package
from scipy.optimize import minimize
import numpy as np

'''#initialize random stasis parameters in the desired intervals
gamma = random.uniform(4,8)
delta = random.uniform(0,3)
alpha = (2/7)*gamma - (1/delta)
delm = random.uniform(1,10)
m0 = delm
fplas = random.uniform(0.1,1)
f = random.uniform(0.1,1)
Tend = random.uniform(1, 10)
Nspecies = int(random.randint(1, 20))
#PRyMini.stasis_params
print(Nspecies)
#initialize params
params = [gamma,delta,alpha,delm,m0,fplas,f,Tend,Nspecies]

# Initial guess
#x0 = np.array([0.0, 0.0])

# Minimize
res = minimize(chi2_fn, params, method='BFGS')'''